In [9]:
import sqlite3
import pandas as pd
from ddgs import DDGS
from crewai.tools import tool
from crewai import Agent, Task, Crew, Process
from agentics import AG

# ============================================
# STEP 1: One-time database setup
# ============================================
def setup_firm_database(csv_path, db_path='firm_data.db', selected_columns=variables):
    """
    Load your CSV into SQLite database (run this once).
    
    Args:
        csv_path: Path to your 600MB CSV file
        db_path: Path where SQLite database will be created
        selected_columns: List of column names to include (None = include all columns)
    """
    print(f"Loading CSV from {csv_path}...")
    
    # First, read a sample to see available columns
    print("Reading CSV header...")
    sample_df = pd.read_csv(csv_path, nrows=5)
    
    print(f"\nAvailable columns ({len(sample_df.columns)} total):")
    for i, col in enumerate(sample_df.columns, 1):
        print(f"  {i}. {col}")
    
    # If no columns selected, prompt user or use all
    if selected_columns is None:
        print("\n" + "="*70)
        print("No columns specified. You can either:")
        print("1. Use all columns (press Enter)")
        print("2. Or modify the code to pass selected_columns parameter")
        print("="*70)
        response = input("\nPress Enter to use ALL columns, or type 'stop' to exit and modify code: ").strip()
        
        if response.lower() == 'stop':
            print("Exiting. Please modify your code to specify selected_columns.")
            return
        
        selected_columns = sample_df.columns.tolist()
    else:
        # Validate that selected columns exist
        missing = [col for col in selected_columns if col not in sample_df.columns]
        if missing:
            print(f"\n❌ Error: These columns don't exist in CSV: {missing}")
            print("Please check your column names and try again.")
            return
        
        print(f"\n✓ Using {len(selected_columns)} selected columns:")
        for col in selected_columns:
            print(f"  - {col}")
    
    # Filter to selected columns
    sample_df = sample_df[selected_columns]
    
    # Check for duplicates in selected columns
    duplicates = [col for col in selected_columns if selected_columns.count(col) > 1]
    if duplicates:
        print(f"\n⚠ Warning: Found duplicate columns in selection: {set(duplicates)}")
        print("Removing duplicates...")
        selected_columns = list(dict.fromkeys(selected_columns))  # Remove duplicates, preserve order
    
    # Clean column names - remove special characters that SQL doesn't like
    clean_columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_').replace('(', '').replace(')', '').replace('/', '_') 
                    for col in selected_columns]
    
    print(f"\nCleaned column names: {clean_columns[:10]}...")
    
    # Delete existing database to start fresh
    import os
    if os.path.exists(db_path):
        os.remove(db_path)
        print(f"Removed existing database: {db_path}")
    
    # For large files, use chunksize to avoid memory issues
    chunk_size = 50000
    conn = sqlite3.connect(db_path)
    
    first_chunk = True
    chunk_num = 0
    total_rows = 0
    
    print("\nLoading data into database...")
    for chunk in pd.read_csv(csv_path, chunksize=chunk_size, usecols=selected_columns):
        # Apply cleaned column names
        chunk.columns = clean_columns
        
        chunk_num += 1
        total_rows += len(chunk)
        
        if first_chunk:
            chunk.to_sql('firms', conn, if_exists='replace', index=False)
            first_chunk = False
            print(f"\n✓ First chunk loaded. Database columns ({len(clean_columns)} total):")
            print(f"  {', '.join(clean_columns)}")
            print()
        else:
            chunk.to_sql('firms', conn, if_exists='append', index=False)
        
        print(f"  Chunk {chunk_num}: +{len(chunk):,} rows (Total: {total_rows:,})")
    
    conn.close()
    print(f"\n{'='*70}")
    print(f"✓ Database created successfully!")
    print(f"  Location: {db_path}")
    print(f"  Total rows: {total_rows:,}")
    print(f"  Columns: {len(clean_columns)}")
    print(f"{'='*70}")
    
    return clean_columns
    
    # Create indexes for better query performance
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Add indexes on commonly queried columns (adjust based on your CSV columns)
    try:
        cursor.execute("CREATE INDEX IF NOT EXISTS idx_ticker ON firms(ticker)")
        cursor.execute("CREATE INDEX IF NOT EXISTS idx_date ON firms(datadate)")
        print("Indexes created successfully")
    except Exception as e:
        print(f"Note: Could not create indexes - {e}")
    
    conn.commit()
    conn.close()

# ============================================
# STEP 2: Define database query tools
# ============================================
@tool("query_firm_database")
def query_firm_database(sql_query: str) -> str:
    """
    Execute SQL queries on the S&P 500 firm characteristics database.
    
    Available table: 'firms' (contains all firm characteristic data)
    
    Example queries:
    - SELECT * FROM firms WHERE ticker = 'AAPL' LIMIT 10
    - SELECT ticker, mve, bm FROM firms ORDER BY mve DESC LIMIT 20
    - SELECT AVG(mom12m) FROM firms WHERE datadate = '2024-12-31'
    - SELECT ticker, bm, mom12m FROM firms WHERE bm < 0.5 AND mom12m > 0
    
    Args:
        sql_query: A valid SQL SELECT query
    
    Returns:
        Query results as a formatted string
    """
    try:
        conn = sqlite3.connect('firm_data.db')
        df = pd.read_sql_query(sql_query, conn)
        conn.close()
        
        if df.empty:
            return "Query returned no results."
        
        # Format the output nicely
        result = f"Query returned {len(df)} rows:\n\n"
        result += df.to_string(index=False)
        
        return result
    except Exception as e:
        return f"Error executing query: {str(e)}\nPlease check your SQL syntax and try again."

@tool("get_database_schema")
def get_database_schema() -> str:
    """
    Get the schema of the firms database including all column names and types.
    Use this first to understand what data is available for stock selection.
    """
    try:
        conn = sqlite3.connect('firm_data.db')
        cursor = conn.cursor()
        
        # Get column information
        cursor.execute("PRAGMA table_info(firms)")
        columns = cursor.fetchall()
        
        # Get row count
        cursor.execute("SELECT COUNT(*) FROM firms")
        row_count = cursor.fetchone()[0]
        
        # Get sample statistics for key columns
        cursor.execute("SELECT * FROM firms LIMIT 1")
        sample = cursor.fetchone()
        
        conn.close()
        
        schema = f"Database: firm_data.db\nTable: firms\nTotal rows: {row_count:,}\n\nColumns:\n"
        for col in columns:
            schema += f"  - {col[1]} ({col[2]})\n"
        
        return schema
    except Exception as e:
        return f"Error retrieving schema: {str(e)}"

@tool("analyze_stock_characteristics")
def analyze_stock_characteristics(ticker: str) -> str:
    """
    Get detailed analysis of a specific stock's characteristics.
    Use this to evaluate individual stocks for investment potential.
    
    Args:
        ticker: Stock ticker symbol (e.g., 'AAPL', 'MSFT')
    """
    try:
        conn = sqlite3.connect('firm_data.db')
        query = f"SELECT * FROM firms WHERE ticker = '{ticker.upper()}' ORDER BY datadate DESC LIMIT 1"
        df = pd.read_sql_query(query, conn)
        conn.close()
        
        if df.empty:
            return f"No data found for ticker: {ticker}"
        
        # Create a readable analysis
        analysis = f"Stock Analysis for {ticker.upper()}:\n\n"
        analysis += df.to_string(index=False)
        
        return analysis
    except Exception as e:
        return f"Error analyzing stock: {str(e)}"

@tool("screen_stocks_by_criteria")
def screen_stocks_by_criteria(criteria: str) -> str:
    """
    Screen stocks based on fundamental criteria for investment selection.
    This tool helps identify stocks that meet specific investment criteria.
    
    Args:
        criteria: Description of screening criteria (e.g., "high momentum and low book-to-market", 
                 "large cap value stocks", "profitable growth stocks")
    
    The tool will attempt to translate criteria into appropriate SQL filters.
    For precise control, use query_firm_database with exact SQL.
    """
    # This is a helper that returns common screening examples
    examples = f"""
To screen stocks, use query_firm_database with SQL like:

Examples for "{criteria}":

1. High momentum stocks:
   SELECT ticker, mom12m FROM firms WHERE mom12m > 0.1 ORDER BY mom12m DESC LIMIT 20

2. Value stocks (low book-to-market):
   SELECT ticker, bm, mve FROM firms WHERE bm < 0.5 ORDER BY bm ASC LIMIT 20

3. Large cap stocks:
   SELECT ticker, mve FROM firms ORDER BY mve DESC LIMIT 20

4. Combination screening:
   SELECT ticker, mom12m, bm, mve FROM firms 
   WHERE mom12m > 0.05 AND bm < 1.0 AND mve > 12
   ORDER BY mom12m DESC LIMIT 20

Use query_firm_database with the appropriate SQL query to screen stocks.
"""
    return examples

@tool("compare_stocks")
def compare_stocks(tickers: str) -> str:
    """
    Compare multiple stocks side-by-side based on their characteristics.
    
    Args:
        tickers: Comma-separated list of ticker symbols (e.g., "AAPL,MSFT,GOOGL")
    """
    try:
        ticker_list = [t.strip().upper() for t in tickers.split(',')]
        ticker_str = "','".join(ticker_list)
        
        conn = sqlite3.connect('firm_data.db')
        query = f"SELECT * FROM firms WHERE ticker IN ('{ticker_str}') ORDER BY ticker"
        df = pd.read_sql_query(query, conn)
        conn.close()
        
        if df.empty:
            return f"No data found for tickers: {tickers}"
        
        comparison = f"Comparison of {len(ticker_list)} stocks:\n\n"
        comparison += df.to_string(index=False)
        
        return comparison
    except Exception as e:
        return f"Error comparing stocks: {str(e)}"

@tool("web_search")
def web_search(query: str) -> str:
    """Fetch web search results for the given query using DDGS. Use for current market news and context."""
    return str(DDGS().text(query, max_results=10))

# ============================================
# STEP 3: Create specialized investment agents
# ============================================
research_agent = Agent(
    role="Quantitative Research Analyst",
    goal="Analyze firm characteristics data to identify stocks with strong return potential",
    backstory="""
    You are an expert quantitative analyst specializing in factor-based investing. You have access 
    to a comprehensive database of S&P 500 firm characteristics including size, value (book-to-market), 
    momentum, profitability, and other fundamental factors.
    
    Your approach:
    1. First, check the database schema to understand available characteristics
    2. Screen stocks based on proven factors (value, momentum, quality, size)
    3. Analyze specific stocks that meet your criteria
    4. Compare candidates to identify the best opportunities
    
    Remember: Academic research shows that value (low book-to-market), momentum (positive returns), 
    profitability, and quality factors predict future returns. Always explain your reasoning based 
    on these fundamental characteristics.""",
    llm=AG.get_llm_provider(),
    memory=True,
    verbose=True
)

investment_agent = Agent(
    role="Portfolio Manager",
    goal="Make final stock selection decisions and provide actionable investment recommendations",
    backstory="""
    You are a seasoned portfolio manager who synthesizes quantitative analysis with market context 
    to make investment decisions. You take the research analyst's findings and:
    
    1. Validate the stock selections against current market conditions
    2. Consider portfolio construction (diversification, risk management)
    3. Provide clear, actionable recommendations with supporting rationale
    4. Specify position sizes or ranking of investment attractiveness
    
    You aim to construct a portfolio of 5-10 stocks with the highest expected returns based on 
    fundamental characteristics. Your recommendations are clear, concise, and actionable.""",
    llm=AG.get_llm_provider(),
    memory=True,
    verbose=True
)

# ============================================
# STEP 4: Define investment tasks
# ============================================
research_task = Task(
    description="""
    Analyze the S&P 500 firm characteristics database to identify stocks for investment.
    
    User request: {input}
    
    Your process:
    1. Understand what characteristics are available in the database
    2. Screen for stocks with attractive fundamental characteristics (value, momentum, quality, etc.)
    3. Analyze the top candidates in detail
    4. Provide a shortlist of 10-15 stocks with strong return potential
    
    Focus on firms with characteristics that academic research has shown predict future returns.
    """,
    expected_output="A detailed analysis with a shortlist of 10-15 stocks ranked by investment potential, with supporting data on their characteristics",
    agent=research_agent,
    tools=[
        get_database_schema,
        query_firm_database,
        analyze_stock_characteristics,
        screen_stocks_by_criteria,
        compare_stocks
    ],
)

investment_task = Task(
    description="""
    Based on the research analysis, make final stock selection decisions.
    
    User request: {input}
    
    Your process:
    1. Review the research analyst's shortlist
    2. Check current market context using web search if needed
    3. Select the top 5-10 stocks for investment
    4. Provide clear reasoning for each selection
    5. Rank stocks by conviction level (highest to lowest)
    
    Output a final portfolio recommendation with specific stocks and rationale.
    """,
    expected_output="A final portfolio of 5-10 stocks with clear rankings, specific reasons for each selection, and investment thesis",
    agent=investment_agent,
    tools=[
        analyze_stock_characteristics,
        compare_stocks,
        web_search
    ],
    context=[research_task]  # This task uses output from research_task
)

crew = Crew(
    agents=[research_agent, investment_agent],
    tasks=[research_task, investment_task],
    process=Process.sequential,  # Research first, then investment decision
    memory=False,
    verbose=True
)

# ============================================
# STEP 5: Main conversation loop
# ============================================
if __name__ == "__main__":
    # IMPORTANT: Run this ONCE to create your database
    # Uncomment the line below and replace with your CSV path
    setup_firm_database('../green cleaned.csv')
    
    print("=" * 70)
    print("AI INVESTMENT ANALYST - Stock Selection System")
    print("=" * 70)
    print("\nThis system analyzes S&P 500 firm characteristics to recommend stocks.")
    print("\nExample queries:")
    print("  - 'Find me the best value stocks to invest in'")
    print("  - 'What are the top momentum stocks right now?'")
    print("  - 'Build me a portfolio of high-quality growth stocks'")
    print("  - 'Which stocks have the best combination of value and momentum?'")
    print("\nType 'exit' to quit\n")
    print("=" * 70)
    
    conversation = ""
    while user_input := input("\nUSER: ").strip():
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("\nThank you for using the AI Investment Analyst. Goodbye!")
            break
            
        print(f"\n{'=' * 70}")
        print(f"ANALYZING: {user_input}")
        print('=' * 70)
        
        result = crew.kickoff(inputs={"input": conversation + user_input})
        
        print(f"\n{'=' * 70}")
        print("FINAL RECOMMENDATION:")
        print('=' * 70)
        print(f"\n{result}\n")
        
        conversation += f"User>{user_input}\nAI>{result}\n"

Loading CSV from ../green cleaned.csv...
Reading CSV header...

Available columns (165 total):
  1. datadate
  2. permno
  3. comnam
  4. ncusip
  5. shrcd
  6. exchcd
  7. siccd
  8. industry
  9. ticker
  10. gvkey_x
  11. ret_excess
  12. rf
  13. mktcap
  14. be
  15. gvkey_y
  16. fyear
  17. sic2
  18. spi
  19. mve_f
  20. bm
  21. ep
  22. cashpr
  23. dy
  24. lev
  25. sp
  26. roic
  27. rd_sale
  28. rd_mve
  29. agr
  30. gma
  31. chcsho
  32. lgr
  33. acc
  34. pctacc
  35. cfp
  36. absacc
  37. age
  38. chinv
  39. spii
  40. cf
  41. hire
  42. sgr
  43. chpm
  44. chato
  45. pchsale_pchinvt
  46. pchsale_pchrect
  47. pchgm_pchsale
  48. pchsale_pchxsga
  49. depr
  50. pchdepr
  51. chadv
  52. invest
  53. egr
  54. pchcapx
  55. grcapx
  56. grGW
  57. woGW
  58. tang
  59. sin
  60. currat
  61. pchcurrat
  62. quick
  63. pchquick
  64. salecash
  65. salerec
  66. saleinv
  67. pchsaleinv
  68. cashdebt
  69. realestate
  70. divi
  71. divo
  72. obklg
  73


USER:  what are the top momemtum stocks right now? keep in mind the data contains log size, and each column is standardized to have mean 0, variance 1.



ANALYZING: what are the top momemtum stocks right now? keep in mind the data contains log size, and each column is standardized to have mean 0, variance 1.


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 85b0830b-a65e-4d92-aafd-516b43995bc4                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Quantitative Research Analyst                                                                           │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Analyze the S&P 500 firm characteristics database to identify stocks for investment.                       │
│                                                                                                                 │
│      User request: what are the top momemtum stocks right now? keep in mind the data contains log size, and     │
│  each column is standardized to have mean 0, variance 1.                                                        │
│                                                                                                                 │
│      Your process:                                                                                              │
│      1. Understand what characteristics are available in the database                                           │
│      2. Screen for stocks with attractive fundamental characteristics (value, momentum, quality, etc.)          │
│      3. Analyze the top candidates in detail                                                                    │
│      4. Provide a shortlist of 10-15 stocks with strong return potential                                        │
│                                                                                                                 │
│      Focus on firms with characteristics that academic research has shown predict future returns.               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {                                   │
│    "error": {                                                                                                   │
│      "code": 429,                                                                                               │
│      "message": "You exceeded your current quota, please check your plan and billing details. For more          │
│  information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.\n* Quota exceeded for   │
│  metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 15\nPlease retry in      │
│  44.846227459s.",                                                                                               │
│      "status": "RESOURCE_EXHAUSTED",                                                                            │
│      "details": [                                                                                               │
│        {                                                                                                        │
│          "@type": "type.googleapis.com/google.rpc.QuotaFailure",                                                │
│          "violations": [                                                                                        │
│            {                                                                                                    │
│              "quotaMetric": "generativelanguage.googleapis.com/generate_content_free_tier_requests",            │
│              "quotaId": "GenerateRequestsPerMinutePerProjectPerModel-FreeTier",                                 │
│              "quotaDimensions": {                                                                               │
│                "model": "gemini-2.0-flash",                                                                     │
│                "location": "global"                                                                             │
│              },                                                                                                 │
│              "quotaValue": "15"                                                                                 │
│            }                                                                                                    │
│          ]                                                                                                      │
│        },                                                                                                       │
│        {                                                                                                        │
│          "@type": "type.googleapis.com/google.rpc.Help",                                                        │
│          "links": [                                                                                             │
│            {                                                                                                    │
│              "description": "Learn more about Gemini API quotas",                                               │
│              "url": "https://ai.google.dev/gemini-api/docs/rate-limits"                                         │
│            }                                                                                                    │
│          ]                                                                                                      │
│        },                                              

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 39561e57-091b-4167-bec3-dc16e4677a7d                                                                     │
│  Agent: Quantitative Research Analyst                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 85b0830b-a65e-4d92-aafd-516b43995bc4                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

RateLimitError: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {
  "error": {
    "code": 429,
    "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 15\nPlease retry in 44.846227459s.",
    "status": "RESOURCE_EXHAUSTED",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.QuotaFailure",
        "violations": [
          {
            "quotaMetric": "generativelanguage.googleapis.com/generate_content_free_tier_requests",
            "quotaId": "GenerateRequestsPerMinutePerProjectPerModel-FreeTier",
            "quotaDimensions": {
              "model": "gemini-2.0-flash",
              "location": "global"
            },
            "quotaValue": "15"
          }
        ]
      },
      {
        "@type

In [7]:
variables = ['datadate', 'ticker', 'agr', 'bm', 'mom12m', 'mve', 'operprof', 'roeq', 'absacc', 'acc', 'aeavol', 'age', 'baspread', 'BETA', 'bm_ia', 
             'cash', 'cashdebt', 'cashpr', 'cfp', 'cfp_ia', 'chatoia', 'chcsho', 'chempia', 'chfeps', 'chinv', 'chmom', 'chnanalyst', 
             'chpmia', 'chtx', 'cinvest', 'convind', 'currat', 'depr', 'disp', 'divi', 'divo', 'dy', 'ear', 'egr', 'ep', 'fgr5yr', 
             'gma', 'grcapx', 'grltnoa', 'herf', 'hire', 'idiovol', 'ill', 'indmom', 'invest', 'IPO', 'lev', 'mom1m', 'mom36m', 'ms', 
             'mve_ia', 'nanalyst', 'nincr', 'orgcap', 'pchcapx_ia', 'pchcurrat', 'pchdepr', 'pchgm_pchsale', 'pchsale_pchinvt', 
             'pchsale_pchrect', 'pchsale_pchxsga', 'pchsaleinv', 'pctacc', 'pricedelay', 'ps', 'rd', 'rd_mve', 'rd_sale', 'realestate', 
             'retvol', 'roaq', 'roavol', 'roic', 'rsup', 'salecash', 'saleinv', 'salerec', 'secured', 'securedind', 'sfe', 'sgr', 'sin', 
             'sp', 'std_dolvol', 'std_turn', 'stdcf', 'sue', 'tang', 'tb', 'turn', 'zerotrade']